In [1]:
from utils.api import API
from utils.dataloader import DataLoader

In [2]:
TOGETHER_API_KEY = "92a6ac4a8feb39c91b4a3f77219e9c452d927f5f4d543d5969cc11c210795719"
BASE_URL = "https://api.together.xyz"
ALLOWED_SYMPTOMS = ['anxiety', 'concentration problems', 'constipation', 'cough', 'diarrhea', 'fatigue', 'fever', 'headache', 'nausea', 'numbness and tingling', 'pain', 'poor appetite', 'rash', 'shortness of breath', 'trouble drinking fluids', 'vomiting', 'other']

Using the API class provides a modular approach to client definition.

In [3]:
api = API(api_key=TOGETHER_API_KEY, base_url=BASE_URL)
client = api.get_openai()

The DataLoader class loads the csv files. It verifies their existence and provides testing tools.

DataLoader allows for dataframe creation, with uniformization of the labels and column formatting.

In [4]:
dataloader = DataLoader(path="data/")
print(dataloader.list_csv_files())

['data/batch_1_gs.csv', 'data/batch_2_gs.csv', 'data/batch_3_gs.csv', 'data/batch_4_gs.csv', 'data/batch_5_gs.csv', 'data/batch_6_gs.csv', 'data/batch_7_gs.csv', 'data/batch_8_gs.csv', 'data/batch_9_gs.csv', 'data/batch_10_gs.csv']


In [5]:
df = dataloader.get_standardized_dataframe(context_col="Text Data",
                                           target_col="symptom_status_gs",
                                           keep_other_cols=True)

In [6]:
dataloader.check_symptoms_validity(allowed_symptoms=ALLOWED_SYMPTOMS,
                                   symptoms_col="symptom_detail_gs")

Symptoms in dataframe are valid.


In [7]:
df.describe()

,Context,Symptom Status_bd,Detailed Choices_bd,Source_x,Symptom Status_CVD,Detailed Choices_CVD,Source_y,Differences,Adjudication Decision,Pres_Abs_Diffs,Target,symptom_detail_gs,Symptom Status_KS,Detailed Choices_KS
count,550,268,189,550,550,382,550,128,128,25,550,380,282,185
unique,550,2,95,50,2,144,50,31,69,2,2,147,2,77
top,"D: How may I help you?\n\nP: Hi, umm, so I've ...",Positive,Pain,RES0022.txt,Positive,Pain,RES0022.txt,Other,pain,Negative;Positive,Positive,pain,Positive,Pain
freq,1,188,17,17,382,38,17,37,20,17,380,50,185,27
